# Towards Automated Taxnonomy Construction
### This notebook contains the code that generated the graphs in the Illumination Works, LLC "Towards Automated Taxonomy Construction" Blog

Brian Connolly, Sr. Consultant at Illumination Works, LLC

## install relevant libraries

In [1]:
!pip install moz_sql_parser
!pip install pyspark
!pip install pyarrow
!pip install anytree
!pip install subprocess
!pip install pydot
!pip install python-graphviz
!pip install copy

ERROR: Could not find a version that satisfies the requirement subprocess (from versions: none)
ERROR: No matching distribution found for subprocess


ERROR: Could not find a version that satisfies the requirement python-graphviz (from versions: none)
ERROR: No matching distribution found for python-graphviz
ERROR: Could not find a version that satisfies the requirement copy (from versions: none)
ERROR: No matching distribution found for copy


## Import libraries

In [2]:
from scipy import spatial
import re
from moz_sql_parser import parse
import pandas as pd
from scipy import stats
from pyspark.sql.functions import pandas_udf
from ast import literal_eval
import numpy as np
from anytree import Node, RenderTree
import glob
# https://stackoverflow.com/questions/49333295/rendering-a-tree-in-python-using-anytree-and-graphviz-without-merging-common-no
import anytree
from anytree import Node, RenderTree
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings(action = 'ignore')
import gensim
from gensim.models import Word2Vec
# https://stackoverflow.com/questions/56741217/using-anytree-to-convert-json-to-a-image
from anytree.exporter import DotExporter
from anytree.importer import JsonImporter
from anytree import RenderTree
from anytree import Node
from anytree.dotexport import RenderTreeGraph
# https://stackoverflow.com/questions/56741217/using-anytree-to-convert-json-to-a-image
import anytree.exporter as atex
import anytree as at
import json
import pydot
from anytree.dotexport import RenderTreeGraph
from anytree.exporter import DotExporter
import os
import ast
import copy

## Define Functions

### these are the core functions used to get the data ready for analysis
Only the queries in the json data files are used for analysis here, and so it is only important the a record with the key word 'sql' be included in a format like in the following example:

"sql": [
            "SELECT JOURNALalias0.HOMEPAGE FROM JOURNAL AS JOURNALalias0 WHERE JOURNALalias0.NAME = \"journal_name0\" ;"
        ]

In [3]:
def select_udf(x):
    """ 
    function used within pandas apply function to extract fields from (not all) select statements 
    within .json files in the text2sql data set
   
    Parameters
    ------------
        x: str
            json in parsed_sql column in pandas DataFrame build from text2sql .json file

    Return
    -----------
        selects: list
        returns list of select statements or it errors out and returns None
    """ 
    try:
        select = x.get('select')
        selects = []
        if type(select)==list:
            for value in select:
#                 print(value)
                if 'value' in value.keys():
                    val = value.get('value')
                    if type(val)==str:
                        selects += [val]
        elif type(select)==dict and type(select.get('value'))==str:
            selects = [select['value']]
        else:
            if type(select['value']['distinct'])==list:
#                 print(select['value']['distinct'])
                for value in select['value']['distinct']:
                    if 'value' in value.keys():
                        val = value.get('value')
                        if type(val)==str:
                            selects += [val]
            elif type(select['value']['distinct']['value'])==str:
                selects += [select['value']['distinct']['value']] 
        return selects
    except Exception as error:
#        print("select_udf: An exception occurred:", error)
        return None
    
def parse_udf(x):
    """ 
    function used within pandas apply function to parse sql statements 
    within json found within text2sql data set
   
    Parameters
    ------------
        x: str
            json in sql column in pandas DataFrame build from text2sql .json file

    Return
    -----------
        : dict
        sql statements parsed in a .json format
    """ 
    try:
        return parse(x)
    except:    
        return "parse_udf: failed: "+x
    
def groupby_udf(x):
    """ 
    function used within pandas apply function to extract fields from (not all) group by statements 
    within .json files in the text2sql data set
   
    Parameters
    ------------
        x: str
            json in parsed_sql column in pandas DataFrame build from text2sql .json file

    Return
    -----------
        selects: list
        returns list of group by statements or it errors out and returns None
    """ 
    try:
        groupby = x.get('groupby')
        if type(groupby) is list:
            if len(groupby)==0:
                return None
            else:
                groupby = [y['value'] for y in groupby]
#                 print(groupby)
        else:
            groupby = [groupby['value']]
        return groupby
    except Exception as error:
#       print("groupby_udf: An exception occurred:", error)
        return None

def create_trees_2(groupbys,png_directory="graph_png_files"): 
    """ 
    - returns a list of Nodes for tree
    - generates png files of trees for those trees where the top nodes have one field in the graph_png_files sub-directory
   
    Parameters
    ------------
        groupbys: list 
            list of fields in select or groupby statements
        png_directory: str (Optional)
            sub-directory to which png will be written
    Return
    -----------
        nodes: list
            list of Node objects
    """ 

    if not os.path.exists(png_directory):
        os.makedirs(png_directory)
    
    nodes = []
    parent_nodes = []
    for groupby in groupbys: # re-initialize
        parent_nodes += [Node(groupby)]
    for parent_node in parent_nodes:
        if len(parent_node.name)<2:
            for groupby in groupbys: # re-initialize nodes
                nodes += [Node(groupby)]
            groupbys_zipped = [(len(x),x) for x in groupbys]
            nodes = find_children(nodes,groupbys_zipped,parent_node,parent_level=1,max_levels=10)
            ordered_node_list = [(x,len(x.name)) for x in nodes]
            ordered_node_list.sort(key = lambda y: y[1],reverse=True)
            for node,length in ordered_node_list:
                if node.parent is not None:
                    node.name = ','.join(list(set(node.name)-set(node.parent.name)))
            parent_node.name = ','.join(parent_node.name)
    #       print(RenderTree(parent_node))
            base_name = parent_node.name.replace(',','__').replace("*","[WILDCARD]")
            if base_name=="":
                base_name = '_'
            atex.DotExporter(
                parent_node, nodeattrfunc = lambda n : 'label="{}"'.format(n.name),edgeattrfunc = lambda node, child: "dir=none"
            ).to_picture(png_directory+"\\"+base_name+".png")
#       print("")
    return nodes

## Helper functions

In [4]:
def flatten(l):
    """ 
    flattens a list of lists
   
    Parameters
    ------------
        l: list
            list of lists
    Return
    -----------
         : list
            flattened list
    """    
    return [item for sublist in l for item in sublist]

def find_children(nodes,groupbys_zipped,parent_node,parent_level,max_levels):
    """ 
    recursive function that finds the children of parent_node given a list of nodes 
   
    Parameters
    ------------
        nodes: list of Node objects
            current list of nodes over entire tree from which children of parent_node is determined
        groupbys_zipped: list of tuples containing (number of fields in select statement,fields in select or groupby statement)
        parent_node: Node 
            parent node for which you want to find children
        parent_level: int
            level of parent tree over entire tree (top level is parent_level=0)
        max_levels: int
            maximum number of levels that you want to consider for tree
    Return
    -----------
        nodes : list of Node object
            list of child nodes
    """ 
    
    if parent_level>max_levels:
        return nodes
    parent_group = parent_node.name
    groupbys = [x[1] for x in groupbys_zipped]
    for groupby in [x[1] for x in groupbys_zipped]:
        if set(parent_group)<set(groupby):
            child_level = parent_level+1
            children = [x[1] for x in groupbys_zipped if (x[0]==child_level) and (set(parent_group)<set(x[1]))]
            for child in children:
                child_node = find_node_by_name(nodes,child)[1]
                child_node.parent=parent_node
                nodes = find_children(nodes,groupbys_zipped,child_node,child_level,max_levels)
    return nodes
                

def find_node_by_name(list_of_nodes,node_name):
    """ 
    given a list of nodes, return the Node object with the name node_name
   
    Parameters
    ------------
        list_of_nodes: list of Node objects
            list of Node objects - one of which contains a Node with name node_name
        node_name: str
            node name of Node you want to find
    Return
    -----------
        nodes: list
            list of Node objects
    """ 
    
    for inode,node in enumerate(list_of_nodes):
        if node.name==node_name:
            return inode,node
    return None,None

def unique_list_of_lists(my_list):
    """ 
    generic function that, given a list of lists (my_list), returns a unique list of lists
   
    Parameters
    ------------
        my_list: list of lists

    Return
    -----------
        : list
        returns either a unique list, or if it is unable to return the list of some reason, it errors out and returns None
    """ 
    
    try:
        return [list(x) for x in set(tuple(x) for x in my_list)]
    except Exception as error: # handle the exception
        print("unique_list_of_lists: An exception occurred:", error)
        return None

### Run a couple of tests to get some intuition

In [5]:
# [SOME EXPLANATION OF OUTPUT]
groupbys_test1 = [['State'],['State','Governor'],['State','Size']]  
create_trees_2(groupbys_test1,'examples')
#
groupbys_test1 = [['a','b'],['a'],['a','b','c'],['b','c']]  
create_trees_2(groupbys_test1,'examples')
#
groupbys_test2 = [['a','b'],['b'],['a','b','c'],['c'],['a','b','c','d']]    
create_trees_2(groupbys_test2,'examples')
#
selects_test=[["city"],["city","address"],["city","address","square footage"],["city","population"]]
create_trees_2(selects_test,'examples')

[Node("/['city']"),
 Node('/city/address'),
 Node('/city/address/square footage'),
 Node('/city/population')]

## Now ingest the data that is used to create tables in blog, and extract the fields from some of the select statements

In [6]:
# create list of .json files in data set
json_files = [file for file in glob.glob("data\\text2sql-data\\data\\*") if file[-5:]=='.json']
# json_files = json_files[:2] # UNCOMMENT THIS FOR TESTING

In [7]:
# put all the .json files into a single pandas DataFrame
json_df=pd.DataFrame()
for json_file in json_files:
    json_df = pd.concat([json_df,pd.read_json(json_file)])
json_df = json_df.explode('sql')

In [8]:
# extract the select and group by statements
json_df["parsed_sql"] = json_df["sql"].apply(lambda x: parse_udf(x))
json_df['select'] = json_df['parsed_sql'].apply(lambda x: select_udf(x))
json_df['groupby'] = json_df['parsed_sql'].apply(lambda x: groupby_udf(x))

## put unique select statements into a list

In [9]:
selects = unique_list_of_lists(json_df[~json_df['select'].isnull()]['select'])

## create the anytree and put images of trees with various parents in specified png_directory through the create_trees_2 function

In [10]:
create_trees_2(selects,png_directory="graph_png_files")

[Node('/INSTITUTIONalias0.INSTITUTION/A,,,C,L'),
 Node('/FESTIVAL_DETAILalias0.LOCATION/M'),
 Node('/COURSEalias0.TITLE/A,,,M,N'),
 Node('/ARTISTalias0.COUNTRY/,,E,M'),
 Node("/['DETENTIONalias0.DETENTION_SUMMARY']"),
 Node('/ACCOUNTSalias0.NAME/I,,,D'),
 Node("/['BEDTYPE']"),
 Node("/['AIRLINESalias0.COUNTRY']"),
 Node('/DRIVERSalias0.FORENAME/U'),
 Node("/['FLIGHTalias0.AIRLINE_CODE']"),
 Node("/['PAYMENTSalias0.PAYMENT_DATE']"),
 Node("/['PRODUCTSalias0.MANUFACTURER']"),
 Node("/['F_ID', 'SONGalias0.ARTIST_NAME', 'SONGalias0.GENRE_IS']"),
 Node("/['DOCUMENT_TYPESalias0.DOCUMENT_TYPE_CODE']"),
 Node('/RACESalias0.NAME/,,D,T'),
 Node("/['GAS_STATIONalias0.LOCATION']"),
 Node("/['CITEalias0.CITINGPAPERID']"),
 Node("/['DERIVED_TABLEalias0.FRIEND']"),
 Node("/['PROFESSIONALSalias0.EMAIL_ADDRESS']"),
 Node("/['CAMERA_LENSalias0.NAME']"),
 Node("/['THIRD_PARTY_COMPANIESalias0.COMPANY_ID']"),
 Node("/['DOCUMENT_TYPE_DESCRIPTION']"),
 Node("/['GAS_STATIONalias0.LOCATION', 'GAS_STATIONalias0